In [ ]:
# Instalar dependencias necesarias
!pip install yfinance statsmodels scikit-learn --quiet


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Definir el ticker de la acción a analizar
ticker = 'AAPL'  # Puedes cambiarlo por otro símbolo


In [ ]:
# Descargar datos semanales desde 2019 hasta el final de 2024
df = yf.download(ticker, start='2019-01-01', end='2025-01-01', interval='1wk')
df = df[['Close']].dropna()
df.head()


In [ ]:
# Separar datos de entrenamiento (2020-2023) y reales de 2024
df_train = df[(df.index >= '2020-01-01') & (df.index < '2024-01-01')]
df_real_2024 = df[(df.index >= '2024-01-01') & (df.index < '2025-01-01')]


In [ ]:
# Entrenar modelo Holt-Winters con estacionalidad semanal (52 semanas)
modelo = ExponentialSmoothing(df_train['Close'], trend='add', seasonal='add', seasonal_periods=52)
ajuste = modelo.fit()
pronostico_2024 = ajuste.forecast(len(df_real_2024))
pronostico_2024.index = df_real_2024.index


In [ ]:
# Calcular métricas de error
mae = mean_absolute_error(df_real_2024['Close'], pronostico_2024)
rmse = mean_squared_error(df_real_2024['Close'], pronostico_2024, squared=False)
print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')


In [ ]:
# Graficar los últimos 5 años de datos reales y el pronóstico de 2024
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Close'], label='Datos reales (2019-2024)', color='blue')
plt.plot(pronostico_2024.index, pronostico_2024, label='Pronóstico Holt-Winters (2024)', color='red', linestyle='--')
plt.title(f'Pronóstico semanal para 2024 vs Datos reales - {ticker}')
plt.xlabel('Fecha')
plt.ylabel('Precio de cierre')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Graficar residuos del modelo
residuos = df_real_2024['Close'] - pronostico_2024
plt.figure(figsize=(12, 4))
plt.plot(residuos, marker='o')
plt.title('Residuos del modelo Holt-Winters (2024)')
plt.xlabel('Fecha')
plt.ylabel('Error de predicción')
plt.grid(True)
plt.tight_layout()
plt.show()
